<a href="https://colab.research.google.com/github/JesusGuzman21/demo_repo/blob/main/ionq_hackathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install qiskit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 38.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 35.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.4/240.4 KB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 KB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 KB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 51.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 KB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.5/37.5 MB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 KB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 41.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.7/112.7 KB

In [ ]:
import numpy as np
import pickle
import json
import os
import qiskit
from collections import Counter
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit, assemble, Aer
from qiskit.visualization import plot_bloch_multivector, plot_histogram, plot_bloch_vector, array_to_latex, plot_state_qsphere
from math import pi, sqrt

In [ ]:
sim = Aer.get_backend('aer_simulator') 

## Part 1

In [ ]:
def normalize_image(image):
    sum_sq = 0
    for row in image:
        for pixel in row:
            sum_sq += pixel ** 2
            
    return image / np.sqrt(sum_sq)
    
def encode(image):
    image = normalize_image(image)
    n = 10
    qc = QuantumCircuit(n)
    
    # 10 qubits, cada uno tiene 2 estados == 1024 estados a disposicion
    size = len(image)
    initial_state = np.zeros(2**n)
    counter = 0
    
    for i in range(size):
        for j in range(size):
            initial_state[counter] = image[i][j]
            counter += 1
            
    qc.initialize(initial_state)
    qc.save_statevector()
       
    return qc

In [ ]:
def decode(counts):
    size = 28
    reconstruction = np.zeros([size, size])

    binaries = list(counts.keys())
    decimals = list(map(lambda b: int(b, 2), binaries))

    for b in list(counts.keys()):
        d = int(b, 2)
        i = int(np.floor(d / 28))
        j = int(d % 28)
        reconstruction[i][j] = counts[b]
        
    return reconstruction

## Part 2

In [ ]:
from qiskit.circuit.library import TwoLocal
from qiskit import BasicAer, execute
from qiskit.algorithms.optimizers import SPSA
import matplotlib.pyplot as plt


In [ ]:
def parity(bitstring):
  hamming_weight = sum(int(k) for k in list(bitstring))
  return (hamming_weight + 1) % 2

In [ ]:
def label_probability(counts):
  # How many attempts
  shots = sum(counts.values())
  probabilities = {0: 0, 1: 0}

  # Loops results, and adds up labels based on parity and their probability
  for bitstring, val in counts.items():
    label = parity(bitstring)
    probabilities[label] += val / shots

  return probabilities

In [ ]:
def circuit_builder(image, vars):
  encoder = encode(image)
  circuit_var = TwoLocal(10, ['ry', 'rz'], 'cz', reps=1)
  circuit = encoder.compose(circuit_var)

  params = {}
  for i, p in enumerate(circuit_var.ordered_parameters):
    params[p] = vars[i]
    
  circuit.assign_parameters(params, inplace = True)

  return circuit

In [ ]:
some = classification_probability(data[0:4], initial_vars)

In [ ]:
print(some)

[{0: 0.5049189027130156, 1: 0.4950810972869833}, {0: 0.516126209191528, 1: 0.483873790808468}, {0: 0.5093948812938653, 1: 0.49060511870613704}, {0: 0.5072400051774394, 1: 0.4927599948225598}]


In [ ]:
def classification_probability(data, vars):
  # Makes circuit for every data input
  circuits = [circuit_builder(d, vars) for d in data]
  
  # Runs all circuits
  qobj = assemble(circuits)
  results = sim.run(qobj).result()

  # Classifies all
  classification = [label_probability(results.get_counts(c)) for c in circuits]

  return classification

In [ ]:
def cross_entropy_loss(classification, expected):
  p = classification.get(expected) 
  return -np.log(p + 1e-10)

def cost_function(data, labels, vars):
  classifications = classification_probability(data, vars)
  cost = 0

  for i, classification in enumerate(classifications):
    cost += cross_entropy_loss(classification, labels[i])
    
  cost /= len(data)

  return cost

In [ ]:
class OptimizerLog:
    def __init__(self):
        self.evaluations = []
        self.parameters = []
        self.costs = []
    def update(self, evaluation, parameter, cost, _stepsize, _accept):
        self.evaluations.append(evaluation)
        self.parameters.append(parameter)
        self.costs.append(cost)

In [ ]:
# Load data and initial vars
data = np.load('./data.npy')
labels = np.load('./labels.npy')

circuit_var = TwoLocal(10, ['ry', 'rz'], 'cz', reps=1)
initial_vars = np.random.random(circuit_var.num_parameters)

In [ ]:
d = data[0:3]
print(len(data[0:3]))
print((d[0][0][11]))

3
0.0030757400999615533


In [ ]:
log = OptimizerLog()
optimizer = SPSA(maxiter=5, callback=log.update)

def objective_function(vars):
    return cost_function(data, labels, vars)

# Run the optimization
result = optimizer.minimize(objective_function, initial_vars)

opt_var = result.x
opt_value = result.fun

fig = plt.figure()
plt.plot(log.evaluations, log.costs)
plt.xlabel('Steps')
plt.ylabel('Cost')
plt.show()